In [1]:
from evaluation.evaluate import (
    get_froc_vals,
    convert_mm_to_pixel,
    get_F1_scores,
)
from monkey.data.data_utils import open_json_file, extract_id
import os
from pprint import pprint

/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
|2024-11-12|10:56:22.694| [WARNING] /home/u1910100/miniconda3/envs/tiatoolbox/lib/python3.11/site-packages/umap/distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-f

In [7]:
GROUND_TRUTH_DIRECTORY = (
    "/home/u1910100/Downloads/Monkey/annotations/json"
)
FOLD = 4
model_name = "efficientunetb0_seg_bm"
PREDICT_DIR = f"/home/u1910100/Documents/Monkey/local_output/{model_name}/Fold_{FOLD}"
SPACING_LEVEL0 = 0.24199951445730394

split_info = open_json_file(
    "/home/u1910100/Documents/Monkey/patches_256/wsi_level_split.json"
)

val_wsi_files = split_info[f"Fold_{FOLD}"]["test_files"]

inflamm_sum_score = 0.0
inflamm_sum_f1 = 0.0
lymph_sum_score = 0.0
lymph_sum_f1 = 0.0
mono_sum_score = 0.0
mono_sum_f1 = 0.0

for wsi_name in val_wsi_files:
    wsi_name_without_ext = os.path.splitext(wsi_name)[0]
    wsi_id = extract_id(wsi_name)

    gt_inf_cells = open_json_file(
        json_path=os.path.join(
            GROUND_TRUTH_DIRECTORY,
            f"{wsi_id}_inflammatory-cells.json",
        )
    )
    gt_lymphocytes = open_json_file(
        json_path=os.path.join(
            GROUND_TRUTH_DIRECTORY, f"{wsi_id}_lymphocytes.json"
        )
    )
    gt_monocytes = open_json_file(
        json_path=os.path.join(
            GROUND_TRUTH_DIRECTORY, f"{wsi_id}_monocytes.json"
        )
    )

    location_detected_inflamm = os.path.join(
        PREDICT_DIR, f"{wsi_id}_detected-inflammatory-cells.json"
    )
    result_detected_inflamm = open_json_file(
        json_path=location_detected_inflamm,
    )
    result_detected_inflamm = convert_mm_to_pixel(
        result_detected_inflamm
    )

    location_detected_lymphocytes = os.path.join(
        PREDICT_DIR, f"{wsi_id}_detected-lymphocytes.json"
    )
    result_detected_lymphocytes = open_json_file(
        json_path=location_detected_lymphocytes,
    )
    result_detected_lymphocytes = convert_mm_to_pixel(
        result_detected_lymphocytes
    )

    location_detected_monocytes = os.path.join(
        PREDICT_DIR, f"{wsi_id}_detected-monocytes.json"
    )
    result_detected_monocytes = open_json_file(
        json_path=location_detected_monocytes,
    )
    result_detected_monocytes = convert_mm_to_pixel(
        result_detected_monocytes
    )

    inflamm_froc = get_froc_vals(
        gt_inf_cells,
        result_detected_inflamm,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    inflamm_f1 = get_F1_scores(
        gt_inf_cells,
        result_detected_inflamm,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    lymph_froc = get_froc_vals(
        gt_lymphocytes,
        result_detected_lymphocytes,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    lymph_f1 = get_F1_scores(
        gt_lymphocytes,
        result_detected_lymphocytes,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    mono_froc = get_froc_vals(
        gt_monocytes,
        result_detected_monocytes,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    mono_f1 = get_F1_scores(
        gt_monocytes,
        result_detected_monocytes,
        radius=int(7.5 / SPACING_LEVEL0),
    )

    # pprint(wsi_id)
    # pprint(f"FROC score: {inflamm_froc['froc_score_slide']}")
    inflamm_sum_score += inflamm_froc["froc_score_slide"]
    inflamm_sum_f1 += inflamm_f1["F1"]

    lymph_sum_score += lymph_froc["froc_score_slide"]
    lymph_sum_f1 += lymph_f1["F1"]

    mono_sum_score += mono_froc["froc_score_slide"]
    mono_sum_f1 += mono_f1["F1"]

pprint(
    f"Average Inflamm FROC = {inflamm_sum_score / len(val_wsi_files)}"
)
pprint(f"Average Inflamm F1 = {inflamm_sum_f1 / len(val_wsi_files)}")
pprint(
    f"Average Lymphocytes FROC = {lymph_sum_score / len(val_wsi_files)}"
)
pprint(
    f"Average Lymphocytes F1 = {lymph_sum_f1 / len(val_wsi_files)}"
)
pprint(
    f"Average Monocytes FROC = {mono_sum_score / len(val_wsi_files)}"
)
pprint(f"Average Monocytes F1 = {mono_sum_f1 / len(val_wsi_files)}")

'Average Inflamm FROC = 0.2455808143884294'
'Average Inflamm F1 = 0.5764051924829452'
'Average Lymphocytes FROC = 0.2864015638967551'
'Average Lymphocytes F1 = 0.4947060244773249'
'Average Monocytes FROC = 0.14807456345268638'
'Average Monocytes F1 = 0.3564056939700898'
